In [7]:
pip install pandas

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 10.8 MB 1.6 MB/s eta 0:00:01
     |████████████████████████████████| 5.3 MB 12.5 MB/s eta 0:00:01
     |████████████████████████████████| 347 kB 15.2 MB/s eta 0:00:01
     |████████████████████████████████| 509 kB 17.0 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [34]:
import pandas as pd

# Load data
nifty = pd.read_csv("Niftweekly.csv")
sensex = pd.read_csv("SensexWeekly.csv")

for df in [nifty, sensex]:
    df.rename(columns={'Change %': 'Change'}, inplace=True)
    df.drop(columns=['Open', 'High', 'Low'], inplace=True)
    df['Date'] = pd.to_datetime(df['Date'], format="%d-%m-%Y")
    df['Change'] = df['Change'].str.replace('%', '', regex=False).astype(float)
    df['Price'] = df['Price'].str.replace(',', '', regex=False).astype(float)
    df['Year'] = df['Date'].dt.year
print(nifty.head())
print(sensex.head())
print(nifty.shape)
print(sensex.shape)


        Date     Price     Vol.  Change  Year
0 2025-06-08  25141.40  887.79M    0.55  2025
1 2025-06-01  25003.05    1.67B    1.02  2025
2 2025-05-25  24750.70    2.71B   -0.41  2025
3 2025-05-18  24853.15    1.68B   -0.67  2025
4 2025-05-11  25019.80    2.07B    4.21  2025
        Date     Price    Vol.  Change  Year
0 2025-06-08  82515.14  37.83M    0.40  2025
1 2025-06-01  82188.99  71.50M    0.91  2025
2 2025-05-25  81451.01  74.32M   -0.33  2025
3 2025-05-18  81721.08  47.02M   -0.74  2025
4 2025-05-11  82330.59  70.04M    3.62  2025
(1328, 5)
(1842, 5)


In [36]:

nifty_dip_weeks = (nifty['Change'] < -3).sum()
sensex_dip_weeks = (sensex['Change'] < -3).sum()

print(f"Nifty: Weeks with fall in 25 Years > 3%: {nifty_dip_weeks}")
print(f"Sensex: Weeks with fall in 35 Years> 3%: {sensex_dip_weeks}")

Nifty: Weeks with fall in 25 Years > 3%: 132
Sensex: Weeks with fall in 35 Years> 3%: 231
